In [13]:
from enum import IntEnum, Enum
import random
from collections import defaultdict
from typing import NamedTuple, List


class HandScore:
    def __init__(self, hand_class, hand_class_qual, starting_ranks):
        self.hand_class: HandClass = hand_class
        self.hand_class_qual: List[Ranks] = hand_class_qual
        self.starting_ranks: List[Ranks] = starting_ranks
        hand_class_qual_hex_str = "0" * 5 + "".join(f"{elt:x}" for elt in self.hand_class_qual)
        hand_class_qual_hex_str = hand_class_qual_hex_str[-5:]
        starting_ranks_hex_str = "".join(f"{elt:x}" for elt in self.starting_ranks)
        self.score = int(f"{self.hand_class}{hand_class_qual_hex_str}{starting_ranks_hex_str}", 16)

    def __repr__(self):
        return f"{self.hand_class.name}{self.hand_class_qual}{self.starting_ranks}"
        
    def __lt__(self, other):
        return self.score < other.score
        
    def __eq__(self, other):
        return self.score == other.score


class HandClass(IntEnum):
    HIGH_CARD = 1
    ONE_PAIR = 2
    TWO_PAIR = 3
    THREE_OF_A_KIND = 4
    STRAIGHT = 5
    FLUSH = 6
    FULL_HOUSE = 7
    FOUR_OF_A_KIND = 8
    STRAIGHT_FLUSH = 9


class Suits(Enum):
    CLUBS = 1
    DIAMONDS = 2
    HEARTS = 3
    SPADES = 4


class Ranks(IntEnum):
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    JACK = 11
    QUEEN = 12
    KING = 13
    ACE = 14

    
class Deck:
    """
    Standard 52-card deck
    13 ranks in each of the four suits: clubs (♣), diamonds (♦), hearts (♥) and spades (♠)
    """
    def __init__(self):
        self.deck = self._init_deck()
        
    def __len__(self):
        return len(self.deck)
    
    def __str__(self):
        return str(self.deck[::-1])
        
    def _init_deck(self):
        deck = []
        for suite in Suits:
            for rank in Ranks:
                deck.append(Card(suite, rank))
        return deck

    def shuffle(self):
        random.shuffle(self.deck)
    
    def draw(self):
        return self.deck.pop()
        
class Card:
    def __init__(self, suite, rank):
        self.suite = suite
        self.rank = rank
        
    def __lt__(self, other):
        return self.rank < other.rank
    
    def __eq__(self, other):
        return self.rank == other.rank

    def __repr__(self):
        return f"{self.rank.name}_{self.suite.name}"


class Player:
    def __init__(self, game):
        self.starting_hand = []
        self.game = game
        
    def add_card(self, card):
        self.starting_hand.append(card)
        
    def get_hand(self):        
        return Hand(self.starting_hand, self.game.get_board())
    
    def get_score(self):
        return self.get_hand().score()


class Hand:
    def __init__(self, starting_hand, board):
        self.starting_hand = starting_hand
        self.board = board
        hand = starting_hand[:]
        hand.extend(self.board)
        self.hand = sorted(hand, reverse=True)
        
        self.hand_class = None
        
    def get_cards(self):
        return self.hand[:]
    
    def get_starting_cards(self):
        return self.starting_hand[:]

    def score(self):
        return Score.score(self.hand[:], self.starting_hand[:])

    def __str__(self):
        return str(self.hand)


class Game:
    def __init__(self, player_count):
        self.hand_size = 2
        self.cur_round = 0
        self.score_log = []

        self.board = []
        self.discard = []
        
        self.deck = Deck()
        self.players = [Player(self) for _ in range(player_count)]        
        self.deck.shuffle()

    def _draw_cards(self):
        if self.cur_round == 0:
            for _ in range(self.hand_size):
                for player in self.players:
                    player.add_card(self.deck.draw())
        elif self.cur_round == 1:
            self.discard.append(self.deck.draw())
            for _ in range(3):
                self.board.append(self.deck.draw())
        else:
            self.discard.append(self.deck.draw())
            self.board.append(self.deck.draw())

    def play_round(self):
        self._draw_cards()
        self.score_log.append([player.get_score() for player in self.players])
        self.cur_round += 1

    def play(self):
        while self.cur_round <= 3:
            self.play_round()

    def get_starting_hands(self):
        return [sorted(player.starting_hand[:], reverse=True) for player in self.players]
    
    def get_board(self):
        return sorted(self.board[:], reverse=True)

    def get_discard(self):
        return sorted(self.discard[:], reverse=True)
    
    def get_score_log(self):
        return self.score_log

    
class Score:    
    @staticmethod
    def _get_highest(cards):
        return cards[0]

    @staticmethod
    def _get_seq(cards):
        best_seq = [cards[0]]
        cur_seq = [cards[0]]
        
        for card_idx in range(1, len(cards)):
            if cards[card_idx].rank == cards[card_idx - 1].rank:
                continue

            elif cards[card_idx].rank == cards[card_idx - 1].rank - 1:
                cur_seq.append(cards[card_idx])
                if len(cur_seq) > len(best_seq):
                    best_seq = cur_seq

            else:
                cur_seq = [cards[card_idx]]

        return best_seq
        
    @staticmethod
    def score(cards, starting_cards):
        scores = defaultdict(list)
        cards = sorted(cards, reverse=True)
        starting_cards = [card.rank for card in sorted(starting_cards, reverse=True)]

        rank_counter = defaultdict(int)
        suite_counter = defaultdict(int)
        for card in cards:
            rank_counter[card.rank] += 1
            suite_counter[card.suite] += 1
            
        for rank in sorted(rank_counter):
            if rank_counter[rank] == 4:
                scores[HandClass.FOUR_OF_A_KIND].append(HandScore(HandClass.FOUR_OF_A_KIND, [rank], starting_cards))
            elif rank_counter[rank] == 3:
                scores[HandClass.THREE_OF_A_KIND].append(HandScore(HandClass.THREE_OF_A_KIND, [rank], starting_cards))
            elif rank_counter[rank] == 2:
                scores[HandClass.ONE_PAIR].append(HandScore(HandClass.ONE_PAIR, [rank], starting_cards))
        
        if len(scores[HandClass.ONE_PAIR]) > 1:
            scores[HandClass.TWO_PAIR].append(HandScore(HandClass.TWO_PAIR, [scores[HandClass.ONE_PAIR][-1].hand_class_qual[0], scores[HandClass.ONE_PAIR][-2].hand_class_qual[0]], starting_cards))
            
        if scores[HandClass.ONE_PAIR] and scores[HandClass.THREE_OF_A_KIND]:
            scores[HandClass.FULL_HOUSE].append(HandScore(HandClass.FULL_HOUSE, [scores[HandClass.THREE_OF_A_KIND][-1].hand_class_qual[0], scores[HandClass.ONE_PAIR][-1].hand_class_qual[0]], starting_cards))

        for suite in suite_counter:
            if suite_counter[suite] >= 5:
                flush_cards = [card.rank for card in cards if card.suite == suite]
                scores[HandClass.FLUSH].append(HandScore(HandClass.FLUSH, flush_cards[:5], starting_cards))

        best_seq = Score._get_seq(cards)
        if len(best_seq) >= 5:
            best_seq = best_seq[:5]
            scores[HandClass.STRAIGHT].append(HandScore(HandClass.STRAIGHT, [card.rank for card in best_seq], starting_cards))
        
        if scores[HandClass.STRAIGHT] and scores[HandClass.FLUSH]:
            scores[HandClass.STRAIGHT_FLUSH].append(HandScore(HandClass.STRAIGHT_FLUSH, scores[HandClass.STRAIGHT][-1].hand_class_qual, starting_cards))

        scores[HandClass.HIGH_CARD].append(HandScore(HandClass.HIGH_CARD, starting_cards, starting_cards)) 
            
        scores = {k:v for k, v in scores.items() if v}
        return scores[sorted(scores.keys(), reverse=True)[0]][-1]


In [14]:
g = Game(4)

In [16]:
g.get_starting_hands()

[[], [], [], []]

In [17]:
g.get_board()

[]

In [18]:
g.play()

In [19]:
g.get_starting_hands()

[[ACE_DIAMONDS, EIGHT_CLUBS],
 [FIVE_SPADES, FOUR_CLUBS],
 [JACK_DIAMONDS, TEN_CLUBS],
 [SEVEN_DIAMONDS, TWO_DIAMONDS]]

In [9]:
g.get_score_log()

[[HIGH_CARD[<Ranks.ACE: 14>, <Ranks.TEN: 10>][<Ranks.ACE: 14>, <Ranks.TEN: 10>],
  HIGH_CARD[<Ranks.JACK: 11>, <Ranks.SEVEN: 7>][<Ranks.JACK: 11>, <Ranks.SEVEN: 7>],
  HIGH_CARD[<Ranks.SIX: 6>, <Ranks.THREE: 3>][<Ranks.SIX: 6>, <Ranks.THREE: 3>],
  HIGH_CARD[<Ranks.FIVE: 5>, <Ranks.TWO: 2>][<Ranks.FIVE: 5>, <Ranks.TWO: 2>]],
 [HIGH_CARD[<Ranks.ACE: 14>, <Ranks.TEN: 10>][<Ranks.ACE: 14>, <Ranks.TEN: 10>],
  HIGH_CARD[<Ranks.JACK: 11>, <Ranks.SEVEN: 7>][<Ranks.JACK: 11>, <Ranks.SEVEN: 7>],
  TWO_PAIR[<Ranks.SIX: 6>, <Ranks.THREE: 3>][<Ranks.SIX: 6>, <Ranks.THREE: 3>],
  ONE_PAIR[<Ranks.TWO: 2>][<Ranks.FIVE: 5>, <Ranks.TWO: 2>]],
 [HIGH_CARD[<Ranks.ACE: 14>, <Ranks.TEN: 10>][<Ranks.ACE: 14>, <Ranks.TEN: 10>],
  HIGH_CARD[<Ranks.JACK: 11>, <Ranks.SEVEN: 7>][<Ranks.JACK: 11>, <Ranks.SEVEN: 7>],
  TWO_PAIR[<Ranks.SIX: 6>, <Ranks.THREE: 3>][<Ranks.SIX: 6>, <Ranks.THREE: 3>],
  ONE_PAIR[<Ranks.TWO: 2>][<Ranks.FIVE: 5>, <Ranks.TWO: 2>]],
 [ONE_PAIR[<Ranks.SIX: 6>][<Ranks.ACE: 14>, <Ranks.TEN: 1

In [10]:
g.get_score_log()[-1][0]

ONE_PAIR[<Ranks.SIX: 6>][<Ranks.ACE: 14>, <Ranks.TEN: 10>]

In [11]:
total_trials_count = 100000
player_count = 4

result_counter = defaultdict(int)
results = []

for _ in range(total_trials_count):
    g = Game(player_count)
    g.play()
    result = g.get_score_log()[-1]
    results.append(result)
    for res in result:
        result_counter[res.hand_class] += 1


In [12]:
for key in sorted(result_counter):
    print(f"{key.name}: {result_counter[key]/(total_trials_count * player_count)}")


HIGH_CARD: 0.176675
ONE_PAIR: 0.4393675
TWO_PAIR: 0.23608
THREE_OF_A_KIND: 0.0483025
STRAIGHT: 0.0417575
FLUSH: 0.0289475
FULL_HOUSE: 0.0253325
FOUR_OF_A_KIND: 0.0016625
STRAIGHT_FLUSH: 0.001875
